# Temporal Trends

How do crime rates very by neighborhood over time?

Are certain types of crimes more prevelant during particular times of the year?

What's the trend in reporting delay?

Which neighborhoods had the sharpest increase in violent crimes since 2020?